# Control the Franka Panda Robot via Teleoperation.

* **Environment**

| Package       | Version   |
|---------------|-----------|
| gymnasium     | No Need   |
| numpy         | 1.24.0    |
| grpcio        | 1.64.1    |
| pygame        | 2.6.0     |
| Level Name    | Franka_Joystick (controllable level, with or without recording) |
| Level Name    | Franka_Joystick_Replay (replay only, non-controllable) |

* Control the gripper's pose using the controller
* Guide the robot's movement through mocap
* Implement recording and replay functionality to store and replay control inputs, used for demonstration and imitation learning training


## Testing Xbox Controller Input

* Run a test program to confirm that the controller is receiving input signals
* Press the corresponding buttons to print information

In [ ]:
import os
import sys
import time

current_file_path = os.path.abspath('')
project_root = os.path.dirname(current_file_path)

if project_root not in sys.path:
    sys.path.append(project_root)

from orca_gym.devices.xbox_joystick import XboxJoystickManager

def test_xbox_controller():
    try:
        joystick_manager = XboxJoystickManager()
        joystick_names = joystick_manager.get_joystick_names()
        joysticks = [joystick_manager.get_joystick(name) for name in joystick_names]
        while True:
            joystick_manager.update()      
            for i, joystick in enumerate(joysticks):
                state = joystick.get_state()
                print("Controller:", joystick_names[i])
                print("Buttons:", state["buttons"])
                print("Axes:", state["axes"])
                print("Hats:", state["hats"])
            time.sleep(1)
    except KeyboardInterrupt:
        joystick_manager.close()
        print("Controller closed")

if __name__ == '__main__':
    test_xbox_controller()


## Testing Keyboard Input

To accommodate different operating systems, the Pygame library is used to implement keyboard input detection. When running the program, a small window will appear, and you need to focus on this window for key inputs to be captured. Press the corresponding keys (A/W/S/D/Up/Down, etc.) and observe the printed output.

In [ ]:
import os
import sys
import time

current_file_path = os.path.abspath('')
project_root = os.path.dirname(current_file_path)

if project_root not in sys.path:
    sys.path.append(project_root)

from orca_gym.devices.keyboard import KeyboardInput

def test_keyboard_controller():
    try:
        controller = KeyboardInput()
        while True:
            controller.update()            
            state = controller.get_state()
            print("Keys:", state)
            if state["Esc"]:
                raise KeyboardInterrupt("Esc key pressed")
                break
            time.sleep(1)
    except KeyboardInterrupt:
        controller.close()
        print("Controller closed")

if __name__ == '__main__':
    test_keyboard_controller()

## Control the Franka Panda Robot Arm

Apply offsets to the mocap point inputs based on the controller state

* **Using Xbox Controller:**
    * Left joystick controls parallel movement of the gripper
    * Right joystick controls pitch and yaw of the gripper
    * LT/RT controls forward and backward extension of the gripper
    * LB/RB controls rotation of the gripper
    * A/B controls opening and closing of the gripper

 In this example, the operations will be recorded and stored in a .h5 file (refer to the implementation of the `register_env` function). In the next step, we will replay these operations.


In [ ]:
import os
import sys
import time

current_file_path = os.path.abspath('')
project_root = os.path.dirname(current_file_path)

if project_root not in sys.path:
    sys.path.append(project_root)


import gymnasium as gym
from gymnasium.envs.registration import register
from datetime import datetime
from envs.orca_gym_env import ActionSpaceType

TIME_STEP = 0.005

def register_env(grpc_address, control_freq=20):
    print("register_env: ", grpc_address)
    gym.register(
        id=f"XboxControl-v0-OrcaGym-{grpc_address[-2:]}",
        entry_point="envs.franka_control.franka_joystick_env:FrankaPandaEnv",
        kwargs={'frame_skip': 1,   
                'reward_type': "dense",
                'action_space_type': ActionSpaceType.CONTINUOUS,
                'action_step_count': 0,
                'grpc_address': grpc_address, 
                'agent_names': ['Panda'], 
                'time_step': TIME_STEP,
                'control_freq': control_freq},
        max_episode_steps=sys.maxsize,
        reward_threshold=0.0,
    )

def continue_training(env):
    observation, info = env.reset(seed=42)
    while True:
        start_time = datetime.now()

        action = env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)

        elapsed_time = datetime.now() - start_time
        if elapsed_time.total_seconds() < TIME_STEP:
            time.sleep(TIME_STEP - elapsed_time.total_seconds())

    

if __name__ == "__main__":
    """
    An example of an OSC (Operational Space Control) motion algorithm controlling a Franka robotic arm.
    Level: Franka_Joystick_osc
    Differences from Franka_Joystick:
    1. Motor control uses torque output (moto) instead of setting joint angles.
    2. Torque calculation is based on the OSC algorithm.
    3. The mocap point can move freely and is not welded to the site; the pulling method is not used.
    """
    try:
        grpc_address = "localhost:50051"
        print("simulation running... , grpc_address: ", grpc_address)
        env_id = f"XboxControl-v0-OrcaGym-{grpc_address[-2:]}"

        # RecordState controls the recording of the simulation data
        register_env(grpc_address, 20)

        env = gym.make(env_id)        
        print("Starting simulation...")

        continue_training(env)
    except KeyboardInterrupt:
        print("Simulation stopped")        
        env.close()

## Replay the Previous Operations

Now, open the `Franka_Joystick_Replay` level in OrcaStudio. This level is identical to `Franka_Joystick`, with the main difference being the removal of the `Panda_Mocap` node and the `Mujoco Equality` components from the `Panda` node. In fact, if you prefer, you can avoid opening a new level and instead disable or remove the relevant components in the Env environment implementation by calling the Mujoco API (refer to `envs/franka_control/franka_joystick_env.py` for details).

* Run the following code to see the Panda robot arm perform the same actions you previously executed.


In [ ]:
if __name__ == "__main__":
    try:
        grpc_address = "localhost:50051"
        print("simulation running... , grpc_address: ", grpc_address)
        env_id = f"XboxControl-v0-OrcaGym-{grpc_address[-2:]}"

        # RecordState controls the recording of the simulation data
        register_env(grpc_address, RecordState.REPLAY, 'xbox_control_record.h5')

        env = gym.make(env_id)        
        print("Starting simulation...")

        asyncio.run(continue_training(env))
    except KeyboardInterrupt:
        print("Simulation stopped")
        env.close()